In [6]:

import numpy as np
import torch

In [8]:


!pip install flask_cors
!pip install pymongo
!pip install dotenv
!pip install pandas
!pip install plotly
!pip install json
!pip install langchain_groq

!pip install jellyfish
!pip install markdown2



  Using cached flask_cors-6.0.1-py3-none-any.whl (13 kB)



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 800.2/800.2 kB 4.2 MB/s eta 0:00:00
  Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


...


[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 217.1/217.1 kB 4.5 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached markdown2-2.5.4-py3-none-any.whl (49 kB)


In [2]:

from transformers import BertTokenizer, AutoModel

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)

In [5]:
def embed_tokens(input_ids, attention_mask):
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    # Use the [CLS] token representation for sentence-level embedding
    embeddings = outputs.last_hidden_state[:, 0, :]  # Shape: [batch_size, hidden_dim]
    return embeddings

In [6]:
all_embeddings = torch.load("embeddings.pt")

C:\Users\shrus\AppData\Local\Temp/ipykernel_8880/64800262.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  all_embeddings = torch.load("embeddings.pt")


In [7]:

embeddings = all_embeddings.numpy()

In [8]:
import json

# Load JSON data
with open('data_for_vectordb.json', 'r') as file:
    data = json.load(file)
    

cve_ids = [item['id'] for item in data]
cve_ids = np.array(cve_ids)

In [9]:
from pinecone import Pinecone

pc = Pinecone(api_key="2f5652db-c1bc-4529-a189-84e3602bf46c")
index = pc.Index("cveindex")

In [10]:
import time

batch_size = 100  # Try 100 or 200
vectors = []

start = time.time()

for i, cve_id in enumerate(cve_ids):
    if cve_id:
        vectors.append({
            "id": str(cve_id),
            "values": embeddings[i].tolist()
        })
        
        # Once batch is full or last item, upsert
        if len(vectors) == batch_size or i == len(cve_ids) - 1:
            index.upsert(vectors=vectors, namespace="ns1")
            vectors = []  # Reset batch

print(f"Done in {(time.time() - start)/60:.2f} minutes.")


Done in 19.97 minutes.


In [11]:
query = "An authenticated user may be able to misuse parameters to inject arbitrary operating system commands into mySCADA myPRO versions 8.25.0 and prior."
     

In [12]:

query_encoded_inputs = tokenizer(query, padding=True, truncation=True, return_tensors='pt')

In [13]:
query_token_ids = query_encoded_inputs['input_ids']
query_attention_masks = query_encoded_inputs['attention_mask']

In [14]:
query_embeddings = embed_tokens(query_token_ids, query_attention_masks)

C:\Users\shrus\anaconda3\lib\site-packages\torch\nn\modules\module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [15]:
query_vector = query_embeddings.tolist()

In [16]:
query_vector = query_vector[0]

In [17]:
response = index.query(
    namespace="ns1",
    vector=query_vector,
    top_k=5,
    include_values=True,
)

In [18]:

for res in response['matches']:
    print(res['id'])
     

CVE-2022-0999
CVE-2023-29169
CVE-2023-29150
CVE-2023-28384
CVE-2023-28716
